In [1]:
import pandas as pd
import numpy as np
import collections
import operator #
import matplotlib.pyplot as plt
from itertools import islice
%matplotlib inline

In [2]:
#Load the dataset
data = pd.read_csv("/Users/Rohit/Desktop/Data/test.txt", sep="\t")
mydict = {}
data.shape

(3614506, 5)

In [3]:
data.head() #Check

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,479,family guy,2006-03-01 16:01:20,NaN,NaN
1,479,also sprach zarathustra,2006-03-02 14:48:55,NaN,NaN
2,479,family guy movie references,2006-03-03 22:37:46,1.0,http://www.familyguyfiles.com
3,479,top grossing movies of all time,2006-03-03 22:42:42,1.0,http://movieweb.com
4,479,top grossing movies of all time,2006-03-03 22:42:42,2.0,http://www.imdb.com


In [4]:
queries = data.Query.unique()
len(queries) #No of unique queries

1244496

In [5]:
total = 0
for i in data.Query:
    if i in mydict: 
        mydict[i] += 1
    else: 
        mydict[i] = 1
    total+=1

In [6]:
mydict

{'girls licking': 2,
 'petro nissan pathfinder': 1,
 'www.cheats for runescape': 1,
 'smakos.com': 1,
 'tour for singles': 6,
 'copper cutter': 3,
 'www.vrbo.com': 16,
 'druids': 22,
 'soccer camps': 3,
 'plans for zeus chapter in saint seiya': 4,
 'www.friedappliences.com': 1,
 'mtbjobseekers': 1,
 'customer sevice phone number': 1,
 'tattoos of harley davidson chicks': 1,
 'www.sensecosmetics.com': 1,
 'everette e mills inmate': 2,
 'www.ymcaatl.org': 1,
 'nasty dirty jokes': 1,
 'bigbraintattoos': 1,
 'a 1913581822427m109798067y': 1,
 'lynn creek marina': 2,
 'women blouse white': 2,
 'www.wpec.com': 1,
 'tayra banks': 1,
 'golden plains credit union': 1,
 'probhation': 1,
 'aaa realtors': 4,
 'britneyspearsporn': 1,
 'beach channel high school ged': 1,
 'free racing games': 5,
 'airlines.com': 8,
 'seinna.org': 1,
 'warren and sons': 1,
 'burning of an abraision on the penile glans': 1,
 'aspen theatre': 1,
 'preschool printables': 6,
 'connicelli hynuda': 1,
 'www.scotchwoodfarms.

In [7]:
#Verifying dictionary is correct
sum =0
users =""
for k,v in mydict.items():
    sum+=v   
print(sum) #Should be same as shape[0]

3614506


In [8]:
mydict

{'girls licking': 2,
 'petro nissan pathfinder': 1,
 'www.cheats for runescape': 1,
 'smakos.com': 1,
 'tour for singles': 6,
 'copper cutter': 3,
 'www.vrbo.com': 16,
 'druids': 22,
 'soccer camps': 3,
 'plans for zeus chapter in saint seiya': 4,
 'www.friedappliences.com': 1,
 'mtbjobseekers': 1,
 'customer sevice phone number': 1,
 'tattoos of harley davidson chicks': 1,
 'www.sensecosmetics.com': 1,
 'everette e mills inmate': 2,
 'www.ymcaatl.org': 1,
 'nasty dirty jokes': 1,
 'bigbraintattoos': 1,
 'a 1913581822427m109798067y': 1,
 'lynn creek marina': 2,
 'women blouse white': 2,
 'www.wpec.com': 1,
 'tayra banks': 1,
 'golden plains credit union': 1,
 'probhation': 1,
 'aaa realtors': 4,
 'britneyspearsporn': 1,
 'beach channel high school ged': 1,
 'free racing games': 5,
 'airlines.com': 8,
 'seinna.org': 1,
 'warren and sons': 1,
 'burning of an abraision on the penile glans': 1,
 'aspen theatre': 1,
 'preschool printables': 6,
 'connicelli hynuda': 1,
 'www.scotchwoodfarms.

In [9]:
#Find the most common queries in this list
commonQueries = sorted(mydict.items(), key=lambda x:-x[1])[:100]
commonQueries

[('-', 98554),
 ('google', 32396),
 ('yahoo', 13344),
 ('ebay', 12949),
 ('yahoo.com', 8733),
 ('mapquest', 8680),
 ('google.com', 8139),
 ('myspace', 7653),
 ('myspace.com', 7099),
 ('www.google.com', 4255),
 ('www.yahoo.com', 4196),
 ('internet', 3854),
 ('http', 3117),
 ('weather', 2884),
 ('ebay.com', 2772),
 ('map quest', 2695),
 ('my space', 2545),
 ('pogo', 2474),
 ('american idol', 2429),
 ('www.myspace.com', 2416),
 ('ask jeeves', 2228),
 ('bank of america', 2129),
 ('hotmail.com', 1924),
 ('msn', 1887),
 ('mycl.cravelyrics.com', 1838),
 ('dictionary', 1793),
 ('hotmail', 1705),
 ('msn.com', 1618),
 ('mapquest.com', 1544),
 ('craigslist', 1541),
 ('my space.com', 1435),
 ('free porn', 1414),
 ('tattoo', 1399),
 ('yahoo mail', 1399),
 ('walmart', 1397),
 ('.com', 1373),
 ('ask.com', 1360),
 ('southwest airlines', 1342),
 ('white pages', 1273),
 ('www.letstalkhonestly.com', 1247),
 ('porn', 1235),
 ('maps', 1227),
 ('mailbox', 1213),
 ('www.google', 1167),
 ('pogo.com', 1154),
 

In [15]:
#Given the top choices, we're going to try to look at some patterns in common between the users 
#that used the most frequented query
#Linear reg wouldn't work because its very sensitive to outliers, which is common in modern data
#Noticed most mispelled/unclear queries don't get a click.

In [207]:
#Checking acceptable similarity check for word level check (Max 1 word difference)
def threshold(value, total):
    if value < (total-2)/total:
        return value
    else:
        return 1
    
def simCheck(s1,s2):
    ls = len(s1.split(" "))
    lo = len(s2.split(" "))
    if lo == ls:
        l = ls
        flag = 0
        if len(s1) < len(s2):
            s = s1
            o = s2
        else:
            s = s2
            o = s1
        
        for word in s.split(" "):
            for other in o.split(" "):
                if word == other:
                    flag = flag +1 
                    print(word)
        simfactor = flag / l
        print(flag)
        print(2* l)
        return threshold(simfactor, 2 * l) #Percentage of similarity between two sentences
    else:
        return 0
        

In [208]:
s1 = "This is not awesone"
s2 = "This is not awesome"
simCheck(s1,s2)

This
is
not
3
8


1

In [224]:
for i in mydict.Query:
    print(simcheck(i,[j, for every j in mydict.keys]))

SyntaxError: invalid syntax (<ipython-input-224-fcc9d479ad97>, line 2)